# Step 1: Import data from Repo and Merge to one Dataset

In [23]:
!pip install holidays
import pandas as pd
import holidays

# Define the URLs for the additional datasets
kiwo_url = 'https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv'
wetter_url = 'https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv'
umsatzdaten_gekuerzt_url = 'https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv'
ferien= '/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/ferien_schleswig_holstein_2012_2020.csv'

# Load the datasets
kiwo = pd.read_csv(kiwo_url, delimiter=',')
wetter = pd.read_csv(wetter_url, delimiter=',')
umsatzdaten_gekuerzt = pd.read_csv(umsatzdaten_gekuerzt_url, delimiter=',')
ferien_df = pd.read_csv(ferien, delimiter=',')
# Merge datasets with a left join to retain all rows from wetter
wetter_umsatzdaten = pd.merge(wetter, umsatzdaten_gekuerzt, on="Datum", how="left")

# Add KiWo flag
wetter_umsatzdaten['KiWo'] = wetter_umsatzdaten['Datum'].isin(kiwo['Datum']).astype(int)

# Load Ferien dataset



# Read the ferien_schleswig_holstein_2012_2020.csv file
ferien_df = pd.read_csv('ferien_schleswig_holstein_2012_2020.csv')

# Convert Startdatum and Enddatum to datetime
ferien_df['Startdatum'] = pd.to_datetime(ferien_df['Startdatum'], format='%d.%m.%Y')
ferien_df['Enddatum'] = pd.to_datetime(ferien_df['Enddatum'], format='%d.%m.%Y')

# Create a new column in the existing DataFrame for Ferien
wetter_umsatzdaten['Ferien'] = None

# Convert the date column in the existing DataFrame to datetime if not already
wetter_umsatzdaten['Datum'] = pd.to_datetime(wetter_umsatzdaten['Datum'])

# Iterate through the ferien_df and update the Ferien column in existing_df
for index, row in ferien_df.iterrows():
    mask = (wetter_umsatzdaten['Datum'] >= row['Startdatum']) & (wetter_umsatzdaten['Datum'] <= row['Enddatum'])
    wetter_umsatzdaten.loc[mask, 'Ferien'] = row['Ferienart']

# Save the updated DataFrame to a new CSV file

print(wetter_umsatzdaten.head())
# Merge the datasets
#wetter_umsatzdaten_kiwo = pd.merge(wetter_umsatzdate, wetter_umsatzdaten, on="Datum", how="left")
# Convert the 'Datum' column to datetime
wetter_umsatzdaten['Datum'] = pd.to_datetime(wetter_umsatzdaten['Datum'])

# Define the range of years
years = range(2012, 2019 + 1)

# Create a list to store the holidays
holidays_sh = []

# Loop through each year and get the holidays for Schleswig-Holstein
for year in years:
    for date, name in holidays.Germany(years=year, state='SH').items():
        holidays_sh.append((date, name))

# Create a set of holiday dates
holiday_dates = set(date for date, name in holidays_sh)

# Add a new column 'Holiday' to the wetter_umsatzdaten dataframe
wetter_umsatzdaten['Holiday'] = wetter_umsatzdaten['Datum'].apply(lambda x: 1 if pd.to_datetime(x).date() in holiday_dates else 0)

# Replace NaN in Umsatz with 0
wetter_umsatzdaten['Umsatz'] = wetter_umsatzdaten['Umsatz'].fillna(0)

# Replace NaN in Warengruppe with 0
wetter_umsatzdaten['Warengruppe'] = wetter_umsatzdaten['Warengruppe'].fillna(0).astype(int)

# Create a new column 'ID' to match the sample submission format (YYMMDDW)
wetter_umsatzdaten['ID'] = (
    wetter_umsatzdaten['Datum'].dt.strftime('%y%m%d').astype(str) + 
    wetter_umsatzdaten['Warengruppe'].astype(str)
).astype(int)

# Print the first few rows to verify the new ID format
print(wetter_umsatzdaten[['Datum', 'Warengruppe', 'ID']].head())

# Save merged dataset directly
wetter_umsatzdaten.to_csv("wetter_umsatzdaten_kiwo_hol_id.csv", index=False)

print("Merge complete. Data saved for subsequent processing.")


# Load the merged dataset
wetter_umsatzdaten_kiwo = pd.read_csv("../0_DataPreparation/wetter_umsatzdaten_kiwo_hol_id.csv")

# Step 1: Inspect the data
print("Data Overview:")
print(wetter_umsatzdaten_kiwo.tail())
print("Shape of Data:", wetter_umsatzdaten_kiwo.shape)
print("Missing values per column:")
print(wetter_umsatzdaten_kiwo.isnull().sum())




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
       Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  id  \
0 2012-01-01         8.0      9.8250                   14        58.0 NaN   
1 2012-01-02         7.0      7.4375                   12         NaN NaN   
2 2012-01-03         8.0      5.5375                   18        63.0 NaN   
3 2012-01-04         4.0      5.6875                   19        80.0 NaN   
4 2012-01-05         6.0      5.3000                   23        80.0 NaN   

   Warengruppe  Umsatz  KiWo Ferien  
0          NaN     NaN     0   None  
1          NaN     NaN     0   None  
2          NaN     NaN     0   None  
3          NaN     NaN     0   None  
4          NaN     NaN     0   None  
       Datum  Warengruppe       ID
0 2012-01-01            0  1201010
1 2012-01-02            0  1201020
2 2012-01-03            0  1201030
3 2012-01-04            0  1201040
4 2012-01-05


# Step 2: Feature Engineering

In [16]:
# Convert 'Datum' to datetime and add a weekday column
wetter_umsatzdaten_kiwo['Datum'] = pd.to_datetime(wetter_umsatzdaten_kiwo['Datum'], errors='coerce')
wetter_umsatzdaten_kiwo['Wochentag'] = wetter_umsatzdaten_kiwo['Datum'].dt.day_name()
wetter_umsatzdaten_kiwo['Is_Weekend'] = wetter_umsatzdaten_kiwo['Datum'].dt.weekday.isin([5, 6]).astype(int)


# Add a temperature category
def classify_temperature(temp):
    if temp < 0:
        return -2  # Very Cold
    elif 0 <= temp <= 10:
        return -1  # Cold
    elif 10 < temp <= 20:
        return 1   # Warm
    else:
        return 2   # Very Warm

wetter_umsatzdaten_kiwo['Temperature_Category'] = wetter_umsatzdaten_kiwo['Temperatur'].apply(classify_temperature)


# Add a Beaufort scale categorization function
def classify_wind_speed_beaufort(wind_speed):
    if wind_speed <= 0.2:
        return 0  # Calm
    elif wind_speed <= 1.5:
        return 1  # Light Air
    elif wind_speed <= 3.3:
        return 2  # Light Breeze
    elif wind_speed <= 5.4:
        return 3  # Gentle Breeze
    elif wind_speed <= 7.9:
        return 4  # Moderate Breeze
    elif wind_speed <= 10.7:
        return 5  # Fresh Breeze
    elif wind_speed <= 13.8:
        return 6  # Strong Breeze
    elif wind_speed <= 17.1:
        return 7  # Near Gale
    elif wind_speed <= 20.7:
        return 8  # Gale
    elif wind_speed <= 24.4:
        return 9  # Strong Gale
    elif wind_speed <= 28.4:
        return 10  # Storm
    elif wind_speed <= 32.6:
        return 11  # Violent Storm
    else:
        return 12  # Hurricane

wetter_umsatzdaten_kiwo['Windgeschwindigkeit_Beaufort'] = wetter_umsatzdaten_kiwo['Windgeschwindigkeit'].apply(classify_wind_speed_beaufort)

# Add a column for Rain_Status based on Wettercode
def map_rain_status(wettercode):
    rain_status_mapping = {
        0: 0,
        20: 0,
        21: 0,
        61: 1,
        63: 1,
        65: 1,
        95: 2
    }
    return rain_status_mapping.get(wettercode, -1)  # Default to -1 for unknown codes

wetter_umsatzdaten_kiwo['Rain_Status'] = wetter_umsatzdaten_kiwo['Wettercode'].apply(map_rain_status)

#Add acolumn for Cloud_Status based on Bewölkung
def map_cloud_status(bewoelkung):
    if bewoelkung < 1:
        return 0  # sunny
    elif bewoelkung <= 3:
        return 1 # clear
    elif bewoelkung <= 6:
        return 2  # Partly Cloudy
    elif bewoelkung <= 7:
        return 3  # Cloudy
    else:
        return 4  # totally Cloudy

wetter_umsatzdaten_kiwo['Cloud_Status'] = wetter_umsatzdaten_kiwo['Bewoelkung'].apply(map_cloud_status)


# Generate rows for all Warengruppe categories where Warengruppe is 0
unique_warengruppen = wetter_umsatzdaten_kiwo['Warengruppe'].unique()
rows_with_zero_warengruppe = wetter_umsatzdaten_kiwo[wetter_umsatzdaten_kiwo['Warengruppe'] == 0]

# Expand rows for each Warengruppe
expanded_rows = []
for _, row in rows_with_zero_warengruppe.iterrows():
    for warengruppe in unique_warengruppen:
        if warengruppe != 0:  # Exclude 0 as it's already in the dataset
            new_row = row.copy()
            new_row['Warengruppe'] = warengruppe
            expanded_rows.append(new_row)

# Append expanded rows back to the main dataset
expanded_df = pd.DataFrame(expanded_rows)
wetter_umsatzdaten_kiwo = pd.concat([wetter_umsatzdaten_kiwo, expanded_df], ignore_index=True)

# Perform one-hot encoding for 'Warengruppe'
warengruppe_encoded = pd.get_dummies(wetter_umsatzdaten_kiwo['Warengruppe'], prefix='Warengruppe')
wetter_umsatzdaten_kiwo = pd.concat([wetter_umsatzdaten_kiwo, warengruppe_encoded], axis=1)

# Adjust IDs based on Warengruppe
def adjust_id(row):
    for i in range(1, 7):  # Warengruppe 1 to 6
        if row[f'Warengruppe_{i}']:
            return str(row['ID'])[:-1] + str(i)  # Replace last digit with Warengruppe
    return row['ID'] # Return original ID if no adjustment is needed

# Apply ID adjustments
wetter_umsatzdaten_kiwo['ID'] = wetter_umsatzdaten_kiwo.apply(adjust_id, axis=1)

# Remove entries where Warengruppe 0 is True
wetter_umsatzdaten_kiwo = wetter_umsatzdaten_kiwo[~wetter_umsatzdaten_kiwo['Warengruppe_0']]

# Remove the column for Warengruppe 0
wetter_umsatzdaten_kiwo.drop(columns=['Warengruppe_0'], inplace=True)

# Dynamically create a list of feature columns
feature_columns = [
    'KiWo', 
    'Is_Weekend', 
    'Temperature_Category', 
    'Windgeschwindigkeit_Beaufort', 
    'Rain_Status'
    'Cloud_Status'
] + [col for col in warengruppe_encoded.columns if col != 'Warengruppe_0']

# Step 3: Save Processed Data
wetter_umsatzdaten_kiwo.to_csv("processed_data.csv", index=False)

# Save feature columns to a file for use in the modeling stage
with open("feature_columns.txt", "w") as f:
    f.write("\n".join(feature_columns))

print("Data preparation and characterization complete. Processed data and feature columns saved.")

Data preparation and characterization complete. Processed data and feature columns saved.
